In [ ]:
import numpy as np
import pandas as pd
from xgboost import XGBRegressor
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf
import statistics
from sklearn.model_selection import learning_curve
from sklearn.pipeline import make_pipeline

In [ ]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [ ]:
"""
Function that loads and prepares the dataframe. 

Converts the data into time-series readable.
"""
def initialFormat (filepath, indicatorcode):
    #Reads the file and creates a dataframe from it
    df = pd.read_excel(filepath)
    
    #Choose what to forecast using indicator code
    df_icode = df.loc[df['Indicator Code'] == indicatorcode]
    
    #Dropping these columns as they are not needed for the forecast
    df_icode = df_icode.drop(columns=['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code'])
    
    #Swap axis so it is in the proper format
    df_formatted = df_icode.swapaxes("index", "columns")
    
    #Renaming column name to 'values' to make reference easier
    for col_names in df_formatted.columns:
        name = df_formatted.rename(columns={col_names : "Val"})
        return name
    
    return df_formatted

In [ ]:
"""
Refactored from: 

https://cprosenjit.medium.com/10-time-series-forecasting-methods-we-should-know-291037d2e285

"""

def shift_dataframe(data):
    
    data["Target"] = data.Val.shift(-1)
    data.dropna(inplace=True)
    
    return data

In [ ]:
"""
adapted and modified from: 
https://towardsdatascience.com/time-series-from-scratch-train-test-splits-and-evaluation-metrics-4fd654de1b37
"""

def train_test_split(data):
    
    train = data[:int(len(data)*0.8)]
    test = data[int(len(data) * 0.8):]
    
    return train, test

In [ ]:
"""
line 8 in this function reused from: 
https://stackoverflow.com/questions/26921651/how-to-delete-the-last-row-of-data-of-a-pandas-dataframe
"""

def mean_imputation(data):
    filled = data.fillna(data.mean())
    filled.drop(data.tail(1).index,inplace=True) #remove last row
    
    return filled

In [ ]:
"""
This function is refactored from machine learning mastery to fit my data:

https://machinelearningmastery.com/xgboost-for-time-series-forecasting/
"""
def xgboost_model(train, X_test):
    train = np.asarray(train)
    #Split into X_train and y_train
    X_train, y_train = train[:,0:1], train[:, -1]
    
    model = XGBRegressor(objective='reg:squarederror', n_estimators=5, max_depth=3, learning_rate=0.3)
    model.fit(X_train, y_train, verbose=False)
    
    y_pred = model.predict(np.asarray([X_test]))
    
    return y_train, X_train, y_pred[0]

In [ ]:
"""
This function is refactored from machine learning mastery:

https://machinelearningmastery.com/xgboost-for-time-series-forecasting/

"""

def walk_forward_validation(data):
    predictions = list()
    
    train, test = train_test_split(data)
    
    history = [x for x in train]
    
    for i in range(len(test)):
        
        X_test, y_test = test[i, :-1], test[i, -1]
        
        y_train, X_train ,y_pred = xgboost_model(history, X_test)
        
        predictions.append(y_pred)
        
    return y_train, X_train, test[:, -1], predictions

#9 Lines taken exactly from guide

In [ ]:
"""
This piece of code was reused and slightly modified from: 
https://towardsdatascience.com/xgboost-fine-tune-and-optimize-your-model-23d996fab663

Fitted for my model
"""

def tune_n_estimator(X, y):
    hyperparam = {'max_depth': [3,6,10],
                 'learning_rate': [0.01, 0.05, 0.1, 0.15, 0.2, 0.3],
                 'n_estimators': [5,10,50,100,1000]}
    
    model = XGBRegressor()
    
    gscv = GridSearchCV(estimator=model, param_grid=hyperparam,
                       scoring='neg_mean_squared_error',
                       verbose=1)
    
    gscv.fit(X, y)
    
    print("Best hyperparam: ", gscv.best_params_)

In [ ]:
"""
line 7,8,13 (MDA) adapted and modified from: https://gist.github.com/bshishov/5dc237f59f019b26145648e2124ca1c9

line 12 (MAPE) adapted from: https://www.statology.org/mape-python/
"""

def performance_metrics(y_test, y_pred):
    
    sign1 = np.sign(np.array(y_test[1:]) - np.array(y_test[:-1]))
    sign2 = np.sign(np.array(y_pred[1:]) - np.array(y_pred[:-1]))
    rmse = np.sqrt(metrics.mean_squared_error(y_test, y_pred))
    mape = np.mean(np.abs((y_test - y_pred) / y_test )) *100
    mae = metrics.mean_absolute_error(y_test, y_pred)
    r2 = metrics.r2_score(y_test, y_pred)
    mda = np.mean((sign1 == sign2).astype(int))
    mean = np.mean(y_test)
    si = (rmse/mean)*100
    
    print("RMSE: ", rmse)
    print("MAPE: ", mape)
    print("MAE: ", mae)
    print("Scatter Index: ", si)
    print("MDA: ", mda)
    print("Mean of actual: ", mean)

In [ ]:
"""
Commented out line was the line I used for local testing. Uncommented 'data ='
should be good for universal path

universal path with help from: 

https://stackoverflow.com/questions/50119792/python-import-excel-file-using-relative-path

INDICATOR CODES:

GDP GROWTH = NY.GDP.MKTP.KD.ZG

INFLATION = NY.GDP.DEFL.KD.ZG

UNEMPLOYEMENT MALE = SL.UEM.TOTL.MA.ZS

UNEMPLOYMENT FEMALE = SL.UEM.TOTL.FE.ZS

REAL INTEREST RATE = FR.INR.RINR
"""


#data = initialFormat('/Users/farhanhabibie/Desktop/Farhan Thesis Code /UG-Project-Farhan/Indonesia Macro Dataset.xlsx', 
#                    'NY.GDP.MKTP.KD.ZG')

data = initialFormat(r'./../Indonesia Macro Dataset.xlsx', 'NY.GDP.MKTP.KD.ZG')

filled = mean_imputation(data)
shifted = shift_dataframe(filled)
finalData = shifted.values

In [ ]:
y_train, X_train, actual, predicted = walk_forward_validation(finalData)

In [ ]:
"""
Using X_train and Y_train due to how the target variable works for
time series forecast with XGBoost
"""
#tune_n_estimator(X_train, y_train)

In [ ]:
dtest = data.fillna(data.mean())
dtest.drop(data.tail(1).index, inplace=True)
newindextest = dtest[int(len(data)*0.8):]
newindex = newindextest.index.values.tolist()

In [ ]:
"""
Code written by me, however (line 9) was also used in a previous Introduction to AI Course I partook in,
at City University of London

Link to repository provided: 
https://github.com/LabiKSV/intro-to-ai-farhan-labi/blob/main/Linear%20Regression%20Label%20Encoder.ipynb
"""

df_compare = pd.DataFrame({'Actual' : actual, 'Predicted' : predicted})
df_compare.index = newindex
df_compare.plot(title='Unemployment (M) Actual vs Predicted')
performance_metrics(actual, predicted)

In [ ]:
"""
Calc variance

used this guide for formula: https://www.scribbr.com/statistics/variance/
"""

dataMean = data.dropna().mean()
#print(dataMean)
n_minus_one = len(X_train) - 1
summed = 0
for vals in X_train:
        subtracted = vals - dataMean
        squared = subtracted ** 2
        summed = summed + squared

#print(summed)

varianced = summed / n_minus_one
print("Variance = ", varianced)

#9 lines written by me after reading formula from guide

In [ ]:
plt.plot(data)
plt.title('Real Interest Rate Actual Data')

#2 lines written by me

In [ ]:
"""
Learning Curve to detect overfit/underfit

Method from: 

https://vitalflux.com/learning-curves-explained-python-sklearn-example/amp/
and
https://www.scikit-yb.org/en/latest/api/model_selection/learning_curve.html#:~:text=Learning%20curves%20can%20be%20generated,see%20in%20the%20following%20examples.

"""

pipeline = make_pipeline(XGBRegressor(objective='reg:squarederror', n_estimators=1000, 
                                                        max_depth=6, learning_rate = 0.3))

train_sizes, train_scores, test_scores = learning_curve(estimator=pipeline, X=X_train,
                                                      y=y_train, cv=5,
                                                      train_sizes=np.linspace(0.1, 1.0, 10))

train_mean = np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)
test_mean = np.mean(test_scores, axis=1)
test_std = np.std(test_scores, axis=1)

plt.plot(train_sizes, train_mean, marker='o', label='Training Accuracy')
plt.xlabel('Training Data Size')
plt.plot(train_sizes, test_mean, marker='x', linestyle='--', label='Validation Accuracy')
plt.ylabel('Score')
plt.legend(loc='best')
plt.title('Learning Curve, XGB Univariate Real Interest Rate')

#14 lines taken directly from guide, integrated with my model


In [ ]:
train_mean

In [ ]:
test_mean

In [ ]:
"""
101 Total Lines of code
"""